In [13]:
import pandas as pd
import folium

In [14]:
# importing dataset for all cases in karnataka. The data was webscraped from 
df_cases = pd.read_csv('karnataka_cases_wiki_scrape.csv')
df_cases.head()

,Patient Number,Date Announced,Date Discharged/ Deceased,Origin type,Level,Origin,Location,Sex-Age,Ref(s)
0,1,9-Mar-20,27-Mar,T,1,"Austin, USA via Dubai",Bengaluru,M46,[46]
1,2,10-Mar-20,24-Mar,T2P,2,P#1 - Wife,Bengaluru,F47,[47]
2,3,10-Mar-20,27-Mar,T2P,2,P#1 - Daughter,Bengaluru,F13,[47]
3,4,10-Mar-20,24-Mar,T,1,"Los Angeles, USA via London",Bengaluru,M50,[47]
4,5,12-Mar-20,20-Mar,T,1,Greece,Bengaluru,M26,[7][48]


In [15]:
# Getting the latitude and longitude of the locations
from geopy.geocoders import Nominatim
import warnings
warnings.filterwarnings('ignore')

def get_latlng(loc_name):
    geolocator = Nominatim(user_agent="myGeocoder")
    country= 'India'
    loc = geolocator.geocode(loc_name+','+ country)
    #print(loc_name)
    return loc.latitude, loc.longitude

In [19]:
# creating a freq table dataset
loc_counts = df_cases[['Location']].apply(pd.value_counts)
loc_counts['freq'] = loc_counts.Location.values
loc_counts['Location'] = loc_counts.Location.keys()

In [21]:
# getting the lat long values
loc_counts['geo_coods'] = loc_counts['Location'].apply(lambda i:get_latlng(i))

loc_counts['long'] = loc_counts['geo_coods'].apply(lambda x:list(x)[1])
loc_counts['lat'] = loc_counts['geo_coods'].apply(lambda x:list(x)[0])

In [22]:
loc_counts.head()

,Location,freq,geo_coods,long,lat
Bengaluru,Bengaluru,91,"(12.9791198, 77.5912997)",77.591300,12.979120
Mysuru,Mysuru,71,"(12.3051828, 76.6553609)",76.655361,12.305183
Belagavi,Belagavi,41,"(15.8572666, 74.5069343)",74.506934,15.857267
Kalaburagi,Kalaburagi,20,"(17.166667, 77.083333)",77.083333,17.166667
Vijayapura,Vijayapura,19,"(16.666667, 75.916667)",75.916667,16.666667


In [23]:
# Creating the folium map for karnataka location
folium_map = folium.Map(location=[15.3173, 75.7139],
                        zoom_start=6,
                        tiles="CartoDB dark_matter")
for index, row in loc_counts.iterrows():
    radius = row['freq']/3
    
    if row['freq'] > 30:
        color="#E37222" # tangerine
    else:
        color="#0A8A9F" # tangerine
    
    folium.CircleMarker(location=(row["lat"],
                                  row["long"]),
                        radius=radius,
                        color=color,
                        fill=True).add_to(folium_map)

In [39]:
folium_map

In [24]:
folium_map.save("karnataka.html")

In [10]:
# # This gives the IP location. This isnt very accurate most of the cases but it is automated
# import geocoder
# myloc = geocoder.ip('me').latlng
# print(myloc)

In [25]:
# Alternate way to get current location , this is using hard coding.
# This is the way i used to get the location. Can give the full address here for even more accurate results
myloc = get_latlng('Sobha Dream Acres')
print(myloc) 
#Output (12.9570571, 77.7091042)

(12.9367199, 77.72084939461337)


In [26]:
folium.CircleMarker(location=(myloc),
                        radius=4,
                        color='lightgray',
                        fill=True).add_to(folium_map)

In [13]:
folium_map

In [27]:
# Getting redzones info
df_red = pd.read_csv('red_zones_bangalore.csv')
df_red.head()

,Location
0,Singasandra
1,Hagadur
2,Garudachar Palya
3,Varthuru
4,Vasanth Nagar


In [28]:
# Adding bengaluru, karnataka in order to avoid areas from same names from different parts of the country
df_red['Location'] = df_red['Location'].apply(lambda x: x+', Bengaluru, Karnataka')

In [32]:
df_red['Location'].head()

0         Singasandra, Bengaluru, Karnataka
1             Hagadur, Bengaluru, Karnataka
2    Garudachar Palya, Bengaluru, Karnataka
3            Varthuru, Bengaluru, Karnataka
4       Vasanth Nagar, Bengaluru, Karnataka
Name: Location, dtype: object

In [33]:
df_red['geo_coods'] = df_red['Location'].apply(lambda x: get_latlng(x))

In [34]:
df_red.head()

,Location,geo_coods
0,"Singasandra, Bengaluru, Karnataka","(12.88027485, 77.65475374894244)"
1,"Hagadur, Bengaluru, Karnataka","(12.963343949999999, 77.74728529140302)"
2,"Garudachar Palya, Bengaluru, Karnataka","(12.9938796, 77.7017538)"
3,"Varthuru, Bengaluru, Karnataka","(12.9357953, 77.73197740477164)"
4,"Vasanth Nagar, Bengaluru, Karnataka","(12.988721250000001, 77.58516877601824)"


In [35]:
df_red['long'] = df_red['geo_coods'].apply(lambda x:list(x)[1])
df_red['lat'] = df_red['geo_coods'].apply(lambda x:list(x)[0])

In [75]:
df_red.to_json('red_zone.json',orient='records')

In [36]:
for index, row in df_red.iterrows():
    folium.CircleMarker(location=(row["lat"], row["long"]),
                        radius=17,
                        color='red',
                        fill=True).add_to(folium_map)

In [60]:
folium_map

In [37]:
folium_map.save("karnataka_red.html")